## Deliverable 2. Create a Customer Travel Destinations Map.

In [77]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [78]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,42.6001,-73.9662,9.00,51,2,1.01,clear sky
1,1,Okhotsk,RU,59.3833,143.3000,-8.07,85,0,8.93,clear sky
2,2,Ilhabela,BR,-23.7781,-45.3581,79.48,74,100,2.82,overcast clouds
3,3,Atuona,PF,-9.8000,-139.0333,78.58,76,5,18.03,clear sky
4,4,Avarua,CK,-21.2078,-159.7750,78.80,83,2,3.44,clear sky


In [81]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [82]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Ilhabela,BR,-23.7781,-45.3581,79.48,74,100,2.82,overcast clouds
3,3,Atuona,PF,-9.8000,-139.0333,78.58,76,5,18.03,clear sky
4,4,Avarua,CK,-21.2078,-159.7750,78.80,83,2,3.44,clear sky
10,10,Mocuba,MZ,-16.8392,36.9856,79.25,89,93,2.24,overcast clouds
11,11,Mangrol,IN,21.1167,70.1167,76.12,22,23,7.87,few clouds
14,14,Bathsheba,BB,13.2167,-59.5167,77.00,73,40,11.50,scattered clouds
16,16,Omboue,GA,-1.5746,9.2618,77.14,90,99,5.12,light rain
21,21,Vaini,TO,-21.2000,-175.2000,78.80,94,90,5.75,light rain
24,24,Rikitea,PF,-23.1203,-134.9692,79.45,76,36,11.10,light rain
29,29,Arraial Do Cabo,BR,-22.9661,-42.0278,76.39,85,58,23.89,broken clouds


In [83]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                153
City                   153
Country                153
Lat                    153
Lng                    153
Max Temp               153
Humidity               153
Cloudiness             153
Wind Speed             153
Current Description    153
dtype: int64

In [84]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# There were no empty rows, no need to use dropna()

In [85]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = np.nan
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Ilhabela,BR,79.48,overcast clouds,-23.7781,-45.3581,NaN
3,Atuona,PF,78.58,clear sky,-9.8000,-139.0333,NaN
4,Avarua,CK,78.80,clear sky,-21.2078,-159.7750,NaN
10,Mocuba,MZ,79.25,overcast clouds,-16.8392,36.9856,NaN
11,Mangrol,IN,76.12,few clouds,21.1167,70.1167,NaN
14,Bathsheba,BB,77.00,scattered clouds,13.2167,-59.5167,NaN
16,Omboue,GA,77.14,light rain,-1.5746,9.2618,NaN
21,Vaini,TO,78.80,light rain,-21.2000,-175.2000,NaN
24,Rikitea,PF,79.45,light rain,-23.1203,-134.9692,NaN
29,Arraial Do Cabo,BR,76.39,broken clouds,-22.9661,-42.0278,NaN


In [86]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping")

Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping


In [87]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [88]:
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Ilhabela,BR,79.48,overcast clouds,-23.7781,-45.3581,Ilha Flat Hotel
3,Atuona,PF,78.58,clear sky,-9.8000,-139.0333,Villa Enata
4,Avarua,CK,78.80,clear sky,-21.2078,-159.7750,Paradise Inn
10,Mocuba,MZ,79.25,overcast clouds,-16.8392,36.9856,Phama Lodge
11,Mangrol,IN,76.12,few clouds,21.1167,70.1167,Hotel The Royal Honours
...,...,...,...,...,...,...,...
653,Florianopolis,BR,77.00,scattered clouds,-27.6146,-48.5012,Hotel Boutique Quinta das Videiras
655,Thiruvananthapuram,IN,87.80,haze,8.4833,76.9167,Hotel Residency Tower
657,Noumea,NC,82.40,clear sky,-22.2763,166.4572,Hôtel Le Lagon
659,Suphan Buri,TH,86.00,overcast clouds,14.4742,100.1222,Vasidtee City Hotel


In [89]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [90]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}
</dd><dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [91]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))